In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df=pd.read_csv('/kaggle/input/democratvsrepublicantweets/ExtractedTweets.csv')

In [ ]:
df['Tweet'].loc[2]

In [ ]:
import re
import spacy
import pandas as pd
from concurrent.futures import ProcessPoolExecutor

# Sample DataFrame


# Function for text preprocessing
def preprocess_text(text):
    # Remove retweets
    text = re.sub(r'^RT[\s]+', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove mentions (@username)
    text = re.sub(r'@[^\s]+', '', text)
    
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Lowercasing
    text = text.lower()
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return text

# Load spaCy English language model


# Load spaCy stop words


# Batch processing using parallelism
batch_size = 100  # Adjust based on your data size
num_batches = len(df) // batch_size + 1

with ProcessPoolExecutor() as executor:
    result = list(executor.map(preprocess_text, df['Tweet']))

# Add the cleaned string to the DataFrame
df['cleaned_string'] = result

# Print the cleaned DataFrame
print(df[['Tweet', 'cleaned_string']])


In [ ]:
df['Tweet']

In [ ]:
df['cleaned_string'] = result

# Print the cleaned DataFrame
print(df[['Tweet', 'cleaned_string']])


In [ ]:
import re
import string
import spacy

def preprocess_text(text):
    # Remove retweets
    text = re.sub(r'^RT[\s]+', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove mentions (@username)
    text = re.sub(r'@[^\s]+', '', text)
    
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Lowercasing
    text = text.lower()
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return text

# Example text
tweet_text = 'RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages. \n\nCongress has allocated about $18…'

# Preprocess the text
preprocessed_text = preprocess_text(tweet_text)

print(preprocessed_text)


In [ ]:
df

In [ ]:
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd

In [ ]:
val_data=df

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd
import torch

# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model = model.to('cuda')    # if GPU is available

# Load validation data

val_texts = val_data['cleaned_string'].tolist()
val_labels = val_data['Party'].map({'Democrat': 1, 'Republican': 0}).tolist()  # convert sentiment to numeric

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset for validation
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

val_dataset = ReviewDataset(val_encodings, val_labels)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Predict with the model


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification#, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.optim import AdamW

import time
# Record start time
start_time = time.time()


# Load data

reviews = val_texts
labels = val_labels
# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(reviews, labels, test_size=0.2)

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Create torch dataset
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataloaders
train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model = model.to('cuda')  # if GPU is available

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
import time
from tqdm import tqdm
import torch

# Assuming you have a model, optimizer, and train_loader defined

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
model.to(device)

# Number of epochs
num_epochs = 3

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Record start time
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    
    # Create a tqdm progress bar for the training loader
    tqdm_train_loader = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', dynamic_ncols=True)
    
    total_loss = 0.0
    num_batches = len(train_loader)
    
    for batch in tqdm_train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Update tqdm progress bar
        tqdm_train_loader.set_postfix({'Loss': loss.item()})
    
    # Calculate and display average loss after each epoch
    average_loss = total_loss / num_batches
    print(f'Epoch {epoch + 1}/{num_epochs} - Average Loss: {average_loss:.4f}')

# Save the model
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)


In [ ]:
for epoch in range(num_epochs):
    model.train()
    
    # Create a tqdm progress bar for the training loader
    tqdm_train_loader = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', dynamic_ncols=True)
    
    total_loss = 0.0
    num_batches = len(train_loader)
    
    for batch in tqdm_train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Update tqdm progress bar
        tqdm_train_loader.set_postfix({'Loss': loss.item()})
    
    # Calculate and display average loss after each epoch
    average_loss = total_loss / num_batches
    print(f'Epoch {epoch + 1}/{num_epochs} - Average Loss: {average_loss:.4f}')

# Save the model
model.save_pretrained('sentiment_model_RoBERTa')

# Record end time
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)


In [ ]:
import pickle

In [ ]:
Pkl_Filename = "Pickle_RL_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)